## Gaussian Elimination

In this section we develop Python code to solve linear systems in the most direct way, via **Gaussian Elimination** (or just **elimination**).

**The main concept of elimination:**

* transform the given system to an equivalent system (with same solution) that but that is *easier* to solve.

The transformation comes in the form of a sequence of **row operations** that preserve the solution of the system while making the solution readily available. There are three row operations that are helpful:

1. Exchange the position of two rows/equations.
    * Notation: $R_i \xleftrightarrow{} R_j$
4. Multiply a row/equation by any nonzero number:
    * Notation: $R_i \to k R_i$
6. Replace any row/equation with the sum of itself and a multiple of another row/equation.
   * Notation: $R_i \to R_i + k R_j$



<a id='GE1'></a>
### Example:  Row operations and elimination

Let's look at an example.

$$
\begin{eqnarray*}
x + 2 y - z & = & 3\\
2x + y - 2z & = & 3\\
-3x + y + z & = & -6
\end{eqnarray*}
$$

Written in augmented form:

$$
\begin{equation}
\left[ \begin{array}{rrr|r}
1 & 2 & -1 & 3 \\
2 & 1 & -2 & 3 \\
-3 & 1 & 1 & -6
\end{array}\right]
\end{equation}
$$

Row operations to transform to an equivalent (upper triangular matrix) are

1. $R_2 \to R_2 - 2 R_1$
2. $R_3 \to R_3 + 3 R_1$
3. $R_3 \to R_3 + \dfrac{7}{3} R_2$

and results in the follwoing equivalent system

$$
\begin{equation}
\left[ \begin{array}{rrr|r}
1 & 2 & -1 & 3 \\
0 & -3 & 0 & -3 \\
0 & 0 & -2 & -4
\end{array}\right]
\end{equation}
$$

At this point, back-substitution can be used to arrive at the solution. Rather than do this now, let's build a python routine to execute the elimination process, illustrated above, to a more general $n\times n$ system.  

We'll assign the array the name $A$, so that we can refer to it later.

In [1]:
import sys
import numpy as np
# Define the augmented matrix in the above example.
# We'll use this as a test case for working with nxn systems.
A = np.array([[1,2,-1,3],[2,1,-2,3],[-3,1,1,-6]])
print(A)

[[ 1  2 -1  3]
 [ 2  1 -2  3]
 [-3  1  1 -6]]


We could start performing operations on our array, but instead we will first write a few bits of code that will do each of these operations in a systematic manner.


The goal of elimination is to perform row operations to produce a an equivalent upper-triangular matrix:

$$
\begin{equation}
\left[ \begin{array}{cccc} 1 & * & * & * \\ 0 & 1 & * & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{equation}
$$

where the asterisk denote some entry (may or may not be zero). Let's now generalize this to an $n\times n$ system:  

In [3]:
def elimin(A,b):
# =============================================================================
#     A is an nxn matrix .
#     b is the right-hand-side of Ax = b.
#     indicies counting from 1. Note python uses zero-based indexing
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A

    if m != n:
        print("Error: Matrix must be square")
        return

    x = np.zeros(n)

    B = np.copy(A).astype('float64')
    r = np.copy(b).astype('float64')
    eps = sys.float_info.epsilon

    # visit each column, up to j = n-1
    for j in range(1,n):
        if abs (B[j-1][j-1])<eps:
            print("Zero pivot encountered.")
            return
        # visit each row, from i = j+1 to n
        for i in range(j+1,n+1):
            scale = B[i-1][j-1]/B[j-1][j-1]
            # visit each column in row i, from k = j+1 to n
            for k in range(j+1,n+1):
                B[i-1][k-1]  = B[i-1][k-1] - scale*B[j-1][k-1]
            # visit right-hand-side:
            r[i-1] = r[i-1] - scale*r[j-1]

    # Solve using back-substitution
    # Visit each row
    for i in range(n,0,-1):
        # visit each column after diagonal entry
        for j in range(i+1,n+1):
            r[i-1] = r[i-1]-B[i-1][j-1]*x[j-1]
        x[i-1] = r[i-1]/ B[i-1][i-1]

    return B, x

In [ ]:
elimin(A[0:3,0:3],A[0:3,3])

(array([[ 1.,  2., -1.],
        [ 2., -3.,  0.],
        [-3.,  7., -2.]]),
 array([3., 1., 2.]))

In [4]:
def lu_decomposition(A):
  m = A.shape[0]  # m is number of rows in A
  n = A.shape[1]  # n is number of columns in A

  if m != n:
      print("Error: Matrix must be square")
      return

  L=np.eye(n)
  U=np.copy(A).astype('float64')

  for j in range(n-1):
    for i in range(j+1,n):
      L[i][j]=U[i][j]/U[j][j]
      for k in range(j, n):
        U[i][k]=U[i][k]-L[i][j]*U[j][k]
  return L, U


In [7]:
A = np.array([[1,2,-1],[2,1,-2],[-3,1,-6]])
lu_decomposition(A)


(array([[ 1.        ,  0.        ,  0.        ],
        [ 2.        ,  1.        ,  0.        ],
        [-3.        , -2.33333333,  1.        ]]),
 array([[ 1.,  2., -1.],
        [ 0., -3.,  0.],
        [ 0.,  0., -9.]]))